In [183]:

import helper
import submission

training_data = helper.read_data('./asset/training_data.txt')
classifier_path = './asset/classifier.dat'

In [78]:
## import modules here 
import pandas as pd
import numpy as np

################# training #################

def train(data, classifier_file):# do not change the heading of the function
    pass # **replace** this line with your code    

################# testing #################

def test(data, classifier_file):# do not change the heading of the function
    pass # **replace** this line with your code


In [184]:
import re
def line_parser(line):
    data = {}

    data['wd'] = line.split(':')[0]

    #data['prefix_2'] = data['wd'][0:1]
    data['prefix_3'] = data['wd'][0:2]
    data['prefix_4'] = data['wd'][0:3]
    
    #data['suffix_2'] = data['wd'][-2:]
    data['suffix_3'] = data['wd'][-3:]
    data['suffix_4'] = data['wd'][-4:]
    
    data['ph'] = line.split(':')[1]

    v = []
    for u in data['ph'].split(' '):
        if re.match('^[A-Za-z]+$', u):
            continue
        v.append(u)
    data['v'] = v

    pri_pos = -1
    for index, u in enumerate(v):
        if u.endswith('1'):
            pri_pos = index + 1
        data[u[:-1]] = index + 1;
    data['pri_pos'] = pri_pos

    pri_rvpos = -1 if pri_pos == -1 else len(v) + 1 - pri_pos
    data['pri_rvpos'] = pri_rvpos

    sec_pos = -1
    for index, u in enumerate(v):
        if u.endswith('2'):
            sec_pos = index + 1
    data['sec_pos'] = sec_pos

    sec_rvpos = -1 if sec_pos == -1 else len(v) + 1 - sec_pos
    data['sec_rvpos'] = sec_rvpos

    pos = sec_pos if pri_pos == -1 else pri_pos
    data['pos'] = pos

    rvpos = sec_rvpos if pri_rvpos == -1 else pri_rvpos
    data['rvpos'] = rvpos

    data['v_num'] = 0;
    for u in data['ph']:
        if re.match('[0-2]$', u):
            data['v_num'] += 1;
    return data

def conditional_res_test(arr, condition):
    report = ''
    stress_rv = [0, 0, 0, 0, 0, 0]
    stress = [0, 0, 0, 0, 0, 0]
    for s in arr:
        v_rvpos = line_parser(s)['rvpos']
        v_pos = line_parser(s)['pos']
        stress_rv[v_rvpos] += 1
        stress[v_pos] += 1
    report += '## ' + condition + ' ##' + '\n'
    report += 'stress: ' + str(stress) + '\n top_rate = ' + str(round(max(stress)/sum(stress),2)) + '\n'
    report += 'stress_rv: ' + str(stress_rv) + '\n top_rate_rv = ' + str(round(max(stress_rv)/sum(stress_rv),2)) + '\n'
    report += '---------' + '\n'

    return report


In [185]:
temp = line_parser("BUCKLED:B AH1 K AH0 L D")

In [186]:
def getUsefulLabel(data):
    data
    data.pop('ph')
    data.pop('v')
    data.pop('wd')
    data.pop('pri_rvpos')
    data.pop('rvpos')
    data.pop('sec_pos')
    data.pop('sec_rvpos')
    temp = data['pri_pos']
    data.pop('pri_pos')
    data.pop('pos')
    return data, temp
getUsefulLabel(temp)

({'AH': 2,
  'prefix_3': 'BU',
  'prefix_4': 'BUC',
  'suffix_3': 'LED',
  'suffix_4': 'KLED',
  'v_num': 2},
 1)

In [187]:
features = []
for data in training_data:
    temp = line_parser(data)
    features.append(getUsefulLabel(temp))

In [149]:
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.cross_validation import KFold
import matplotlib.pyplot as plt


encoder = LabelEncoder()
vectorizer = DictVectorizer(dtype=int, sparse=False)

X, y = list(zip(*features))
# print(y)
X = vectorizer.fit_transform(X)

#print(X)
print(vectorizer.feature_names_)
y = encoder.fit_transform(y);

# print(y)

['AA', 'AE', 'AH', 'AO', 'AW', 'AY', 'EH', 'ER', 'EY', 'IH', 'IY', 'OW', 'OY', 'UH', 'UW', 'prefix_3=AA', 'prefix_3=AB', 'prefix_3=AC', 'prefix_3=AD', 'prefix_3=AE', 'prefix_3=AF', 'prefix_3=AG', 'prefix_3=AH', 'prefix_3=AI', 'prefix_3=AJ', 'prefix_3=AK', 'prefix_3=AL', 'prefix_3=AM', 'prefix_3=AN', 'prefix_3=AO', 'prefix_3=AP', 'prefix_3=AQ', 'prefix_3=AR', 'prefix_3=AS', 'prefix_3=AT', 'prefix_3=AU', 'prefix_3=AV', 'prefix_3=AW', 'prefix_3=AX', 'prefix_3=AY', 'prefix_3=AZ', 'prefix_3=BA', 'prefix_3=BB', 'prefix_3=BE', 'prefix_3=BH', 'prefix_3=BI', 'prefix_3=BJ', 'prefix_3=BL', 'prefix_3=BO', 'prefix_3=BR', 'prefix_3=BU', 'prefix_3=BY', 'prefix_3=CA', 'prefix_3=CE', 'prefix_3=CH', 'prefix_3=CI', 'prefix_3=CL', 'prefix_3=CO', 'prefix_3=CR', 'prefix_3=CS', 'prefix_3=CU', 'prefix_3=CW', 'prefix_3=CY', 'prefix_3=CZ', 'prefix_3=DA', 'prefix_3=DB', 'prefix_3=DC', 'prefix_3=DD', 'prefix_3=DE', 'prefix_3=DH', 'prefix_3=DI', 'prefix_3=DJ', 'prefix_3=DL', 'prefix_3=DM', 'prefix_3=DN', 'prefix_3

In [85]:
print(y)

[0 0 0 ..., 0 0 0]


In [188]:
nb = MultinomialNB(alpha=2)
nb.fit(X, y)
nb.score(X, y)

0.80742000000000003

In [189]:
X.shape

(50000, 18734)

In [95]:
n_folds = 10
kf = KFold(n=X.shape[0], n_folds=n_folds, shuffle=True, random_state=42)

def test_Multinomial_NB(train_X, train_y, test_X, test_y, alpha=1, debug = False):
    nb = MultinomialNB(alpha=alpha)
    nb.fit(train_X, train_y)
    train_error = nb.score(train_X, train_y)
    test_error = nb.score(test_X, test_y)
    if debug:
        print('training error:\t{}'.format(train_error))
        print('testing error:\t{}'.format(test_error))
    return train_error, test_error

def cv_MultinomialNB(alpha = 1.0):
    train_error_total = 0
    test_error_total = 0
    for train, test in kf:
        train_X = X[train]
        test_X = X[test]
        train_y = y[train]
        test_y = y[test]
        train_error, test_error = test_Multinomial_NB(train_X, train_y, test_X, test_y, alpha)
        train_error_total += train_error
        test_error_total += test_error
    return train_error_total/n_folds, test_error_total/n_folds
#     print('===================')
#     print('avg. training error:\t{}'.format(train_error_total/n_folds))
#     print('avg. testing error:\t{}'.format(test_error_total/n_folds))

def cv_plot_MultinomialNB():
    cv_res = []
    rng = sorted([0.5] + list(range(0, 10)))
    for i in rng:
        train_error, test_error = cv_MultinomialNB(i)
        cv_res.append([i, train_error, test_error])
    cv_res_arr = np.array(cv_res)
    plt.figure(figsize=(16,9)) 
    plt.title('Error vs. alpha') 
    plot_train, = plt.plot(cv_res_arr[:,0], cv_res_arr[:,1], label='training')
    plot_test, = plt.plot(cv_res_arr[:,0], cv_res_arr[:,2], label='testing')
    plt.legend(handles=[plot_train, plot_test])
    plt.ylim((min(min(cv_res_arr[:,1]), min(cv_res_arr[:,2])) - 0.01, max(max(cv_res_arr[:,1]), max(cv_res_arr[:,2]))+0.01))
    plt.xticks(rng)    
    
cv_plot_MultinomialNB()

/Users/bowenwu/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/naive_bayes.py:699: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = (np.log(smoothed_fc) -


In [190]:
def isConsonant(u):
    consonant = ['P', 'B', 'CH', 'D', 'DH', 'F', 'G', 'HH', 'JH', 'K', 'L', 'M', 'N',
                'NG', 'R', 'S', 'SH', 'T', 'TH', 'V', 'W', 'Y', 'Z', 'ZH']
    try:
        consonant.index(u)
    except:
        return False;
    else:
        return True;

def line_parser_test_data(line):
    data = {}
    print(type(data))
    data['wd'] = line.split(':')[0]

    #data['prefix_2'] = data['wd'][0:1]
    data['prefix_3'] = data['wd'][0:2]
    data['prefix_4'] = data['wd'][0:3]
    
    #data['suffix_2'] = data['wd'][-2:]
    data['suffix_3'] = data['wd'][-3:]
    data['suffix_4'] = data['wd'][-4:]
    
    data['ph'] = line.split(':')[1]

    data['v_num'] = 0;
    index = 1;
    for u in data['ph'].split(' '):
        if isConsonant(u):
             continue;
        else:
            data['v_num'] += 1
            data[u] = index
            index += 1
    data.pop('ph')
    return data



In [191]:
temp = line_parser_test_data('MINING:M AY N IH NG')

<class 'dict'>


In [173]:
temp

{'AH': 2,
 'EY': 1,
 'prefix_3': 'DA',
 'prefix_4': 'DAT',
 'suffix_3': 'ATA',
 'suffix_4': 'DATA',
 'v_num': 2,
 'wd': 'DATA'}

In [192]:
temp = vectorizer.transform(temp)

In [193]:
temp

array([[0, 0, 0, ..., 0, 0, 2]])

In [194]:
nb.predict(temp)

array([0])

In [7]:
import helper
import submission

training_data = helper.read_data('./asset/training_data.txt')
classifier_path = './asset/classifier.dat'
submission.train(training_data, classifier_path)

test_data = helper.read_data('./asset/tiny_test.txt')
prediction = submission.test(test_data, classifier_path)
print(prediction)

AttributeError: 'str' object has no attribute 'length'

In [8]:
line = 'dsfs:dsf'
data = line.split(':')[0]


In [9]:
data

'dsfs'

In [10]:
type(data)

str

In [11]:
len(data)

4

In [12]:
data.length

AttributeError: 'str' object has no attribute 'length'